Что больше всего влияет на отстутствия сотрудников

Цель - просто научиться кодить в питоне, честно

1) Программа загрузки, предобработки и очистки данных
2) Корреляционный анализ
3) Кластерный анализ
4) Визуализация данных
5) Выводы
6) Источники (stack overflow, умные знакомые, курсы на курсере, божья помощь)

Кто: Настя Болотина, Наталия Диасамидзе, Екатерина Шленова, Юля Юрчак


Библиотеки

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import scipy.stats as ss

1) Программа загрузки, предобработки и очистки данных

In [2]:
path = "https://raw.githubusercontent.com/NDi-ze/python-project/master/datasets_660_1258_MFGEmployees4%20(2).csv"

df = pd.read_csv(path)
df.head() #подключили датасет

,EmployeeNumber,Surname,GivenName,Gender,City,JobTitle,DepartmentName,StoreLocation,Division,Age,LengthService,AbsentHours,BusinessUnit
0,1,Gutierrez,Molly,F,Burnaby,Baker,Bakery,Burnaby,Stores,32.028816,6.018478,36.577306,Stores
1,2,Hardwick,Stephen,M,Courtenay,Baker,Bakery,Nanaimo,Stores,40.320902,5.532445,30.165072,Stores
2,3,Delgado,Chester,M,Richmond,Baker,Bakery,Richmond,Stores,48.822047,4.389973,83.807798,Stores
3,4,Simon,Irene,F,Victoria,Baker,Bakery,Victoria,Stores,44.599357,3.081736,70.020165,Stores
4,5,Delvalle,Edward,M,New Westminster,Baker,Bakery,New Westminster,Stores,35.697876,3.619091,0.000000,Stores


Для исследования возьмем следующие факторы, которые могут влиять на время абсенса (очень хочется сказать абсента ибо уже не вывожу, хочу напицца чтоб забыцца)
Gender
City
DepartmentName
Division
Age
LengthService

и есесна оставим EmployeeNumber и AbsentHours, остальные выкидываем:

In [3]:
df = df.drop(['Surname', 'GivenName', 'JobTitle','BusinessUnit', 'StoreLocation'], axis = 1) 
df.info() #выкинули лишнее

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8336 entries, 0 to 8335
Data columns (total 8 columns):
EmployeeNumber    8336 non-null int64
Gender            8336 non-null object
City              8336 non-null object
DepartmentName    8336 non-null object
Division          8336 non-null object
Age               8336 non-null float64
LengthService     8336 non-null float64
AbsentHours       8336 non-null float64
dtypes: float64(3), int64(1), object(4)
memory usage: 521.1+ KB


In [4]:
df.describe()

,EmployeeNumber,Age,LengthService,AbsentHours
count,8336.000000,8336.000000,8336.000000,8336.000000
mean,4168.500000,42.007086,4.782910,61.283978
std,2406.540255,9.939798,2.462990,49.038365
min,1.000000,3.504743,0.012098,0.000000
25%,2084.750000,35.298748,3.575892,19.127590
50%,4168.500000,42.114924,4.600248,56.005808
75%,6252.250000,48.666943,5.623922,94.284692
max,8336.000000,77.938003,43.735239,272.530123


проверка пропущенных значений

In [5]:
df.isnull().values.any()

False

акодируем переменные, которые object (придадим им числовые значения, чтоб считать потом; превратим из обычной шкалы в порядковую кажется)

In [6]:
ffc = ['City', 'Gender', 'DepartmentName', 'Division']
for i in ffc:
    df=pd.concat([df.drop(i, 1), pd.get_dummies(df[i]).mul(1)], axis=1)
df.head()



,EmployeeNumber,Age,LengthService,AbsentHours,Abbotsford,Agassiz,Aiyansh,Aldergrove,Alexis Creek,Alkali Lake,...,Produce,Recruitment,Store Management,Training,Executive,FinanceAndAccounting,HumanResources,InfoTech,Legal,Stores
0,1,32.028816,6.018478,36.577306,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,2,40.320902,5.532445,30.165072,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,3,48.822047,4.389973,83.807798,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,44.599357,3.081736,70.020165,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,5,35.697876,3.619091,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [7]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8336 entries, 0 to 8335
Columns: 276 entries, EmployeeNumber to Stores
dtypes: float64(3), int64(1), uint8(272)
memory usage: 2.4 MB


In [11]:
df.columns.tolist()

['EmployeeNumber',
 'Age',
 'LengthService',
 'AbsentHours',
 'Abbotsford',
 'Agassiz',
 'Aiyansh',
 'Aldergrove',
 'Alexis Creek',
 'Alkali Lake',
 'Armstrong',
 'Ashcroft',
 'Atlin',
 'Avola',
 'Balfour',
 'Bamfield',
 'Barriere',
 'Bear Lake',
 'Beaver Valley',
 'Bella Bella',
 'Black Point',
 'Black Pool',
 'Blue River',
 'Blueberry',
 'Bob Quinn Lake',
 'Boston Bar',
 'Bouchie Lake',
 'Bougie Creek',
 'Bowen Island',
 'Brackendale',
 'Bridge Lake',
 'Britannia Beach',
 'Burnaby',
 'Burns Lake',
 'Cache Creek',
 'Campbell River',
 'Canal Flats',
 'Cassiar',
 'Castlegar',
 'Celista',
 'Chase',
 'Chemainus',
 'Chetwynd',
 'Chief Lake',
 'Chilako River',
 'Chilanko Forks',
 'Chilliwack',
 'Christina Lake',
 'Clearwater',
 'Clinton',
 'Cluculz Lake',
 'Cobble Hill',
 'Comox',
 'Coquitlam',
 'Cortes Island',
 'Courtenay',
 'Cranbrook',
 'Crawford Bay',
 'Creston',
 'Cumberland',
 "D'arcy",
 'Dawson Creek',
 'Dease Lake',
 'Decker Lake',
 'Douglas Lake',
 'Dragon Lake',
 'Duncan',
 'Elkf

тут должны быть гипотезы но я хз че писать (но поробую)
1. на асбсенс влияет пол и возраст
2. влияение локации, департамента, дивизиона и города незначительно

корреляции

коэф Кендалла (значения от -1 до 1)

In [8]:
x = df['AbsentHours']
y = df['Age']
x.corr(y, method = "kendall") 

0.6584462807667683

In [9]:
ss.kendalltau(x, y)

KendalltauResult(correlation=0.6584462807667683, pvalue=0.0)

вывод: корреляция между абсенсом и возрастом есть, p < 0,5

In [16]:
x = df['AbsentHours']
y = df['F']
x.corr(y, method = "kendall") 


0.0769100280055496

In [14]:
ss.kendalltau(x, y)

KendalltauResult(correlation=0.0769100280055496, pvalue=1.775266544307561e-17)